<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Aims" data-toc-modified-id="Aims-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Aims</a></span></li><li><span><a href="#Adapted-Lotka-Volterra-equations-(ver-2)" data-toc-modified-id="Adapted-Lotka-Volterra-equations-(ver-2)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Adapted Lotka-Volterra equations (ver 2)</a></span><ul class="toc-item"><li><span><a href="#model" data-toc-modified-id="model-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>model</a></span></li><li><span><a href="#parameter-explanation" data-toc-modified-id="parameter-explanation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>parameter explanation</a></span></li></ul></li><li><span><a href="#Single-parameter-effect" data-toc-modified-id="Single-parameter-effect-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Single parameter effect</a></span></li><li><span><a href="#Two-parameter-effect" data-toc-modified-id="Two-parameter-effect-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Two parameter effect</a></span></li><li><span><a href="#Three-parameter-effect" data-toc-modified-id="Three-parameter-effect-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Three parameter effect</a></span></li><li><span><a href="#Random-parameter-effect" data-toc-modified-id="Random-parameter-effect-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Random parameter effect</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></div>

In [1]:
using RCall, PyCall, Plots, DataFrames
spy = pyimport("scipy")
itg = pyimport("scipy.integrate")

PyObject <module 'scipy.integrate' from '/Users/Ho_Pok_Man/.julia/conda/3/lib/python3.7/site-packages/scipy/integrate/__init__.py'>

# Aims

1. try Julia 1.3.1 language (R, Python and Julia in single script)  
2. Model exploration  
3. report expected model behaviour with graphs

# Adapted Lotka-Volterra equations (ver 2)

## model

In [2]:
function ebc_LVc(pops, t=0) # preset: rp, rq, Kp, Kq, Tk
    
    p = pops[:1]
    q = pops[:2]
    
    hpq = (Jq-Tk)*q/(Jp*p+Jq*q-Tk*(p+q))
    hqp = 1-hpq
    
    dpdt = rp*p*(1-(p+hpq*q)/Kp)
    dqdt = rq*q*(1-(q+hqp*p)/Kq)

    return (dpdt, dqdt)
end

ebc_LVc (generic function with 2 methods)

## parameter explanation

parameter | notation | detailed explanation
--- | --- | ---
time | t | time (continuous)
population $p$ | p | population size from lineage $p$ at time t
population $q$ | q | population size from lineage $q$ at time t
growth rate of $p$ | rp | instantaneous growth rate of lineage $p$ at time t
growth rate of $q$ | rq | instantaneous growth rate of lineage $q$ at time t
carrying capacity of $p$ | Kp | carrying capacity of lineage $p$ in system
carrying capacity of $q$ | Kq | carrying capacity of lineage $q$ in system
temperature loss per cell | Tk | assumed average instantaneous energy loss per cell due to environmental temperature

# Single parameter effect

# Two parameter effect

# Three parameter effect

# Random parameter effect

# Conclusion